In [8]:
import os
import json
import pandas as pd
import traceback

In [9]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

In [10]:
load_dotenv() #To capture or load from .env file

True

In [11]:
Key = os.getenv('OPEN_API_KEY')

In [14]:
llm = ChatOpenAI(openai_api_key=Key,model_name='gpt-3.5-turbo',temperature=0.8)

In [19]:
from langchain.llms import OpenAI
from PyPDF2 import PdfReader
from langchain.callbacks import get_openai_callback
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate


In [16]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [17]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [20]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [21]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key='quiz',verbose=True)

In [23]:
TEMPLATE2="""
You are an expert {subject} . Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert of the {subject} of the above quiz:
"""

In [24]:
review_prompt = PromptTemplate(input_variables=['subject','quiz'], template=TEMPLATE2)

In [25]:
review_chain = LLMChain(llm=llm,prompt=review_prompt,output_key='review',verbose=True)

In [27]:
generate_review_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables= ['quiz','review'],verbose=True)

In [28]:
filepath = '/Users/aayushaggarwal/Desktop/mcqgen/data.txt'

In [29]:
with open(filepath,'r') as f:
    TEXT = f.read()

In [30]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [31]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [32]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"

In [33]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_review_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [34]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2543
Prompt Tokens:2067
Completion Tokens:476
Total Cost:0.0040525


In [35]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [37]:
quiz=response.get("quiz")

In [38]:
quiz=json.loads(quiz)

In [39]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [40]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life?',
  'Choices': 'a: Physics | b: Biology | c: Chemistry | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'Which theme explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Energy processing | c: Internal regulation | d: Cell structure',
  'Correct': 'a'},
 {'MCQ': "Who coined the term 'machine learning' in 1959?",
  'Choices': 'a: Donald Hebb | b: Raytheon Company | c: Arthur Samuel | d: Tom M. Mitchell',
  'Correct': 'c'},
 {'MCQ': 'What is the main objective of modern-day machine learning?',
  'Choices': 'a: Creating algorithms for stock trading | b: Developing models for data classification | c: Understanding human cognitive processes | d: Predicting future outcomes',
  'Correct': 'd'},
 {'MCQ': 'Which field did machine learning grow out of as a scientific endeavor?',
  'Choices': 'a: Artificial Intelligence | b: Physics | c: Chemistry | d: Geology',
  'Correct': 'a'}]

In [41]:
quiz=pd.DataFrame(quiz_table_data)

In [42]:
quiz.to_csv("machinelearning.csv",index=False)